# Tutorial 3.1. Creating "Binless" Resampling Schemes: Na<sup>+</sup>/Cl<sup>-</sup> Association Simulations
Author: Jeremy Leung, Anthony Bogetti<br>
Email:&nbsp;&nbsp; jml230@pitt.edu

## Introduction

This document is a companion jupyter notebook for first tutorial in the WESTPA 2.0 Suite of Tutorials. It involves running a Na<sup>+</sup>/Cl<sup>-</sup> association simulation using WESTPA's new binless resampler module and other advanced techniques. Upon completion of this tutorial, a user should be able to customize their own binless scheme,  initiate WE simulations from multiple basis and start states, combine multiple independent WESTPA simulations for analysis, and calculate additional post-simulation dataset. This tutorial builds upon the basics explained in the Na<sup>+</sup>/Cl<sup>-</sup> tutorial of the original suite of tutorials on LiveCoMS ([Bogetti et al., 2019](https://www.livecomsjournal.org/index.php/livecoms/article/view/v1i2e10607)). This notebook is designed such that each cell is self-contained. Feel free to jump to the section of choice.

## Learning Objectives
- To create a binless scheme for splitting and merging trajectories based on k-means clustering using the BinlessMapper resampler module. groupsubgroup option of the resampler module;
- To initiate a WE simulation from multiple starting conformations;
- To combine multiple WE simulations for analysis using the <code>w_multi_west</code> tool;
- To perform post-simulation analysis using the <code>w_crawl</code> tool.


## System Requirements
- OpenMM, Scikit-learn and MDTraj are necessary to run this simulation.
    - Note that MDTraj is a dependency of WESTPA 2.0.
- numpy, matplotlib, ipympyl, mdtraj, and nglview are used in this jupyter notebook. 
    - nglview, matplotlib, ipympl, and numpy are optional for visualization purposes.

## Running the Simulation

The files to initialize and run the WESTPA simulation are included with this jupyter notebook for demonstration purposes. **It is <u>not</u> recommended that you run your WE simulations within a Jupyter Notebook.** The simulation will take a while to complete, so feel free to stop it at any stage. Sample completed files for analysis are provided in the `for_analysis/` directory and `w_crawl/` folder.

In [ ]:
import os
import shutil
# Clean up from previous/ failed simulations.
for i in ['west.h5', 'seg_logs', 'traj_segs','istates','get_pcoord.log']:
    try:
        os.remove(i)
    except OSError:
        try:
            shutil.rmtree(i)
        except OSError:
            pass
        
for i in ['seg_logs','traj_segs','istates']:
    os.mkdir(i)

In [ ]:
import westpa
import numpy
from westpa.cli.core import w_init
from argparse import Namespace

# Initializing the System:
# Set some parameters that WESTPA needs to set simulation state.
args = Namespace(rcfile='west.cfg',
                 verbosity='verbose', ## change to 'debug' if you want a more detailed view of what's happening
                                      ## or 'verbose' for some information
                                      ## or 'quiet' for no information at all. 
                 work_manager='threads')

# Update westpa.rc with these
westpa.rc.process_args(args)

# Initialize the simulation using the tstate and bstate files
w_init.initialize(tstates=None, bstates=None, 
                  tstate_file='tstate.file', bstate_file='bstates/bstates.txt', 
                  segs_per_state=5, shotgun=False)

In [ ]:
import westpa
import numpy
from westpa.cli.core import w_run
import westpa.work_managers as work_managers
from argparse import Namespace

# Running the Simulation.
# Set some parameters that WESTPA needs to set simulation state.
args = Namespace(rcfile = 'west.cfg',
                 verbosity = "verbose",
                 work_manager = 'threads')

# Update westpa.rc with these
westpa.rc.process_args(args)
# Prepare work manager
work_managers.environment.process_wm_args(args)

# Launch the simulation
w_run.run_simulation()

## Monitoring the WE Simulation

The following sample code runs <code>w_pdist</code> and <code>plothist</code> to generate a probability distribution.

In [ ]:
import tarfile

# Untar the files for analysis
for file in ['./for_analysis/nacl.hdf5.h5.1.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')

In [ ]:
# Generate the pdist.h5
!{'w_pdist -W ./for_analysis/01/west.h5'}

In [1]:
## WORK IN PROGRESS DOESN'T WORK
# Generate the pdist.h5
from westpa.cli.tools import w_pdist
from argparse import Namespace
from westpa.work_managers import make_work_manager
import westpa.work_managers as work_managers
import westpa

!{'rm pdist.h5'}
# Update westpa.rc with these
rc_args = Namespace(rcfile='west.cfg',
                 verbosity='debug',
                 work_manager='processes',)
westpa.rc.process_args(rc_args)

w = w_pdist.WPDist()
args = Namespace(we_h5filename='./for_analysis/01/west.h5',
                 rcfile='west.cfg',
                 verbosity='debug',
                 work_manager=None,
                 max_queue_length=None,
                 construct_dataset=None,
                 dsspecs=None,
                 first_iter=1,
                 last_iter=None,
                 bins='100',
                 output='pdist.h5',
                 ignore_out_of_range=False,
                 compress=False,
                )


w.process_args(args)
westpa.rc.work_manager = work_manager = make_work_manager()
w.work_manager = westpa.rc.work_manager
w.go()

rm: cannot remove 'pdist.h5': No such file or directory
-- DEBUG     2022-05-20 14:25:55,835 PID 18121        TID 140611665782592     
   from logger "westpa.rc"
   at location /home/jml230/git5/westpa/src/westpa/core/_rc.py:234 [process_config()]
   ::
   config: {'west': {'system': {'driver': 'westpa.core.systems.WESTSystem', 'system_options': {'pcoord_ndim': 1, 'pcoord_len': 51, 'pcoord_dtype': <class 'numpy.float32'>, 'bins': {'type': 'RecursiveBinMapper', 'base': {'type': 'RectilinearBinMapper', 'boundaries': [[0, 2.61, 'inf']]}, 'mappers': [{'type': 'BinlessMapper', 'ngroups': 5, 'ndims': 1, 'group_function': 'group.kmeans', 'at': [5]}]}, 'bin_target_counts': 5}}, 'propagation': {'max_total_iterations': 100, 'max_run_wallclock': 259200, 'propagator': 'executable', 'gen_istates': False}, 'data': {'west_data_file': 'west.h5', 'datasets': [{'name': 'pcoord', 'scaleoffset': 4}], 'data_refs': {'iteration': '$WEST_SIM_ROOT/traj_segs/iter_{n_iter:06d}.h5', 'segment': '$WEST_SIM_ROOT/tra

TypeError: 'NoneType' object does not support item assignment

In [ ]:
%tb

In [ ]:
# Generate the average hist.pdf
!{'plothist evolution -o evol.pdf pdist.h5'}

In [ ]:
# View the average probability distribution
from IPython.display import IFrame, display
filepath = "evol.pdf"
IFrame(filepath, width=700, height=400)

In [ ]:
# Generate the average hist.pdf
!{'plothist average -o avg.pdf pdist.h5 0'}

In [ ]:
# View the average probability distribution
from IPython.display import IFrame, display
filepath = "avg.pdf"
IFrame(filepath, width=700, height=400)

## Analyzing the WE Simulation

### Visualization of the System

We will now take a look at how one of the basis states looks like. The water box is omitted for visibility. Na<sup>+</sup> is red and Cl<sup>-</sup> is blue in the representation.

In [ ]:
import mdtraj
import nglview
system = mdtraj.load('bstates/01/basis.xml', top='common_files/bstate.pdb')
Na = list(range(0,1)) # Na+ is the first atom
Cl = list(range(1,2)) # Cl- is the second atom
both = Na + Cl # Na+ + Cl-
system = system.atom_slice(both)
view = nglview.show_mdtraj(system)
view.representations = [
    {"type": "ball+stick", "params": {
        "sele": ".Na+", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": ".Cl-", "color": "blue"
    }}
]
view.background = 'white'
view

### Visualization of Coverage of the Start States

The following cells plots the multiple start states in a way that allows us to examine  their spacial coverage. The first cell visualizes the system one-by-one after alignment to Na<sup>+</sup>. The second cell plots all the center-of-masses at once. In some frames, Cl<sup>-</sup> might not be visible unless you move the camera.

Do note that many lines are not necessary for Na<sup>+</sup>/Cl</sup>-</sup> as it is spherically symmetrical, and that its center of mass is equal to its coordinate. Extra code is provided so it could be generalized to larger systems.

In [ ]:
import numpy
import mdtraj
import nglview
lst = numpy.loadtxt('bstates/bstates.txt', usecols=2, dtype=str) # Reading basis state names
tpg = 'common_files/bstate.pdb' # Topology File for basis state (Shared between all bstates)
lst = [x + "/basis.xml" for x in lst] # Change path to point to file name

# Reading reference and setup
# There might be some warnings about unconverged rotation matrices because of the system's rotational symmetry

com = [] # list containing all CoM
a = mdtraj.load('bstates/01/basis.xml', top=tpg) # Load the first
a_slice = a.atom_slice([0,1]) # Just Na+
com.append(numpy.squeeze(mdtraj.compute_center_of_mass(a_slice))) # Save CoM of Na+
a_slice = a.atom_slice([0,1]) # Both Na+/Cl-

# Loading and superposing, storing Center of Mass (CoM) to list for heatmap
c = a_slice
for i in lst:
    b = mdtraj.load('bstates/' + i, top=tpg)
    b = b.atom_slice([0,1])
    b.superpose(a_slice, atom_indices=[0])
    c = mdtraj.join([c,b], check_topology=False)
    # Just saving the CoM of Cl-, since Na+ is superimposed
    com.append(numpy.squeeze(mdtraj.compute_center_of_mass(b.atom_slice([1]))))
com = numpy.asarray(com)

# Now displaying it, note that Cl- is not visible in some frames unless you rotate the camera
view2 = nglview.show_mdtraj(c)
view2.representations = [
    {"type": "ball+stick", "params": {
        "sele": ".Na+", "color": "red"
    }},
    {"type": "ball+stick", "params": {
        "sele": ".Cl-", "color": "blue"
    }}
]
view2.center('.Na+')
view2.control.zoom(-1.75)
view2

In [ ]:
# Looking at the coverage of the sstates, assuming you ran the previous cell
# Comment out the next line or install ipypml if you have trouble viewing.
%matplotlib widget 
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import numpy

fig = matplotlib.pyplot.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

img1 = ax.scatter(com[0,0], com[0,1], com[0,2], s=20, marker='s', color='Red', label='Na+')
img2 = ax.scatter(com[1:,0], com[1:,1], com[1:,2], s=50, marker='.', color='Blue', label='Cl-')

# Labels and Titles
ax.set_title("Basis State Structures Coverage")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.legend()

show()

Due to the rotational symmetry, it might be better to look at the Na<sup>+</sup>/Cl<sup>-</sup> atom-to-atom distances instead, which are already precalculated in each pcoord.init file.

In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
lst = numpy.loadtxt('bstates/bstates.txt', usecols=2, dtype=str) # Reading basis state names
lst2 = ["bstates/" + x + "/pcoord.init" for x in lst] # Change path to point to file name

values = numpy.asarray([numpy.loadtxt(x) for x in lst2])

fig2 = matplotlib.pyplot.figure(figsize=(7,7))
ax2 = fig2.subplots()

img3 = ax2.scatter(0, 0, color='Red', label='Na+')
img4 = ax2.scatter(values[:], numpy.zeros(values.shape[0]), color='Blue', label='Cl-')
img5 = ax2.axvline(x=2.6, ymin=0, ymax=1, label='Bound State', linestyle='--')

# Labels and Titles
ax2.set_title("Basis States Structures Coverage (Atom-to-Atom Distances)")
ax2.set_xlabel('Na+ to Cl- distance ($\mathrm{\AA}$)')
ax2.set_ylabel('y-axis')
ax2.legend()
plt.xlim(-1,23)

plt.show()

### Calculating Rates

In [ ]:
import tarfile
from os import symlink

# Untar the files for analysis.
for file in ['./for_analysis/nacl.hdf5.h5.1.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')
try:
    symlink('./for_analysis/01/west.h5', './west.h5')
except FileExistsError:
    print('A different west.h5 exists in the basic_nacl/. Either delete it or rename it to continue.')

In [ ]:
from westpa.cli.tools import w_ipa
import westpa

w = w_ipa.WIPI()
w.main()
w.interface = 'matplotlib'

## Managing Your Simulations

### Combining Multiple Simulation Runs

The following sample code runs <code>w_multi_west</code> to concatenate two runs.

In [ ]:
import tarfile

# Untar the files
for file in ['./for_analysis/nacl.hdf5.h5.1.tar.gz','./for_analysis/nacl.hdf5.h5.2.tar.gz']:
    with tarfile.open(file) as tar_f:
        tar_f.extractall('./for_analysis')

In [ ]:
# Run w_multi_west in the commandline
!{'w_multi_west -m ./for_analysis/ -n 2'}
print('Done!')

In [ ]:
# Check to see if the multi.h5 file exists
from os.path import exists
exists('multi.h5')

### Using <code>w_crawl</code> to calculate post-simulation auxiliary data

The following sample code runs <code>w_crawl</code> to calculate additional observables post-simulation.

In [ ]:
import os
os.chdir('w_crawl')
# Run w_crawl in the commandline
!{'./run_w_crawl.sh'}
os.chdir('../')
print('Done!')

In [ ]:
# Check to see if the example.h5 file exists
from os.path import exists
exists('./w_crawl/crawl.h5')

## Cleaning Up

In [ ]:
# Run the following bash script to revert your tutorial folder to pristine condition.
!{'./1.clean.sh'}